In [1]:
from biocrnpyler import *
import pylab as plt
import numpy as np
from bokeh.layouts import row

import warnings
import bokeh.io
import bokeh.plotting



class TranscriptionSwitch(Mechanism):
    #Set the name and mechanism_type
    def __init__(self, name="transcription_switch", mechanism_type="transcription"):
        Mechanism.__init__(self, name=name, mechanism_type=mechanism_type)
    
    def update_species(self, switch_off, activator, inhibitor, rnap, transcript = None, **keywords):
        

        return [switch_off, transcript, activator, inhibitor, rnap] 
    
    
    def update_reactions(self, switch_off, activator, inhibitor, component, part_id, rnap, transcript = None, **keywords):
        

            
        switch_on = ComplexSpecies([switch_off, activator], name = str(switch_off).replace("_OFF","_ON"))
        A_I_complex = ComplexSpecies([inhibitor, activator],name = str(switch_off).replace("_OFF","")+"_AI")
        
        ktx = component.get_parameter("ktx", part_id = part_id, mechanism = self)
        kleak = component.get_parameter("kleak", part_id = part_id, mechanism = self)
        kon = component.get_parameter("kon", part_id = part_id, mechanism = self)
        koff = component.get_parameter("koff", part_id = part_id, mechanism = self)
        ka = component.get_parameter("ka", part_id = part_id, mechanism = self)
        
        reaction_activation = Reaction(inputs = [switch_off, activator], outputs = [switch_on], 
                            k = kon )
        
        reaction_deactivation = Reaction(inputs = [switch_on, inhibitor], outputs = [switch_off, A_I_complex], 
                            k = koff )
        
        reaction_complex = Reaction(inputs = [activator, inhibitor], outputs = [A_I_complex], k = ka)
        
        reaction_tx = Reaction(inputs = [switch_on, rnap], outputs = [switch_on, transcript, rnap], k = ktx)
        
        reaction_leak = Reaction(inputs = [switch_off, rnap], outputs = [switch_off, transcript, rnap], k = kleak)
        
        return [reaction_activation, reaction_deactivation, reaction_complex, reaction_tx, reaction_leak]
    
    

In [2]:
class Switch(Promoter):
    def __init__(self, name, transcript, activator, inhibitor, rnap="RNAP", **keywords):
        #Set the Regulator
        #Component.set_species(species, material_type = None, attributes = None)
        # is a helper function that allows the input to be a Species, string, or Component.
        
        self.activator = self.set_species(activator, material_type = "rna") 
        self.inhibitor = self.set_species(inhibitor, material_type = "rna")
        self.transcript = self.set_species(transcript, material_type = "rna")
        self.switch_off = self.set_species(str(name)+"_OFF")
        self.rnap = self.set_species(rnap, material_type = "protein")
        
        #Mechanisms are inherited from the Mixture unless set specifically in self.default_mechanisms.
        custom_mechanisms = {"transcription": TranscriptionSwitch()}
        
        #Always call the superclass __init__() with **keywords passed through
        Promoter.__init__(self, name = name, transcript = transcript, mechanisms = custom_mechanisms, **keywords)

    def update_species(self, **keywords):
        #Mechanisms are stored in an automatically created dictionary: mechanism_type --> Mechanism Instance.
        mech_tx = self.mechanisms["transcription"]
        
        species = [] #A list of species must be returned
        species += mech_tx.update_species(switch_off = self.switch_off, transcript = self.transcript, activator = self.activator, inhibitor = self.inhibitor, rnap = self.rnap)
        
        return species

    def update_reactions(self, **keywords):
        mech_tx = self.mechanisms["transcription"]
        
        reactions = [] #a list of reactions must be returned
        reactions += mech_tx.update_reactions(switch_off = self.switch_off, transcript = self.transcript, activator = self.activator, inhibitor = self.inhibitor, 
                                              rnap = self.rnap, component = self, part_id = "Switch", **keywords)
        return reactions

    

class Source(Promoter):
    def __init__(self, name, transcript, rnap="RNAP", **keywords):
        #Set the Regulator
        #Component.set_species(species, material_type = None, attributes = None)
        # is a helper function that allows the input to be a Species, string, or Component
        self.dna = self.set_species(name)
        self.rnap = self.set_species(rnap, material_type = "protein")
        #Mechanisms are inherited from the Mixture unless set specifically in self.default_mechanisms.
        custom_mechanisms = {"transcription": Transcription_MM()}
        
        #Always call the superclass __init__() with **keywords passed through
        Promoter.__init__(self, name = name, transcript = transcript, mechanisms = custom_mechanisms, **keywords)

    def update_species(self, **keywords):
        #Mechanisms are stored in an automatically created dictionary: mechanism_type --> Mechanism Instance.
        mech_tx = self.mechanisms["transcription"]
        
        species = [] #A list of species must be returned
        species += mech_tx.update_species(dna = self.dna, transcript = self.transcript, rnap = self.rnap)
        
        return species

    def update_reactions(self, **keywords):
        mech_tx = self.mechanisms["transcription"]
        
        reactions = [] #a list of reactions must be returned
        reactions += mech_tx.update_reactions(dna = self.dna, transcript = self.transcript, 
                                              rnap = self.rnap, component = self, part_id = "Source", **keywords)
        return reactions    
    
    
    

In [3]:
def GeneletGate(name, out, on_1 = None, on_2 = None, off_1 = None, off_2 = None, typ = "AND"):
    if type(name) != str:
        raise RuntimeError('AND gate name must be a string')
    if typ != "AND" and typ != "OR":
        raise RuntimeError('Gate type, typ, must be AND or OR')
        
    if typ == "AND":
        #source = 164
        source = 170
    elif typ == "OR":
        #source = 130
        source = 102
        
    if off_1 == None:
        off_1 = name + "_I1"
    if off_2 == None:
        off_2 = name + "_I2"
    if on_1 == None:
        on_1 = name + "_A1"
    if on_2 == None:
        on_2 = name + "_A2"
    
    
    
    S1_off = Species(name + "_INP1")
    S2_off = Species(name + "_INP2")
    S3_off = Species(name + "_OUT")
    So1_on = Species(name + "_SOU")

    S1 = Switch(S1_off, transcript = name + "_out_A", activator = on_1, inhibitor = off_1 )
    S2 = Switch(S2_off, transcript = name + "_out_A", activator = on_2, inhibitor = off_2 )
    S3 = Switch(S3_off, transcript = out, activator = name + "_out_A", inhibitor = name + "_out_I" )
    So1 = Source(So1_on, transcript = name + "_out_I")
    
    ic = {str(S1_off)+"_OFF": 500, "rna_"+on_1: 700, "rna_"+off_1: 200, str(S2_off)+"_OFF": 500, "rna_"+on_2: 700, "rna_"+off_2: 200, str(S3_off)+"_OFF": 500,
          "rna_"+name+"_out_A": 0, "rna_"+name+"_out_I": 0, str(So1_on):source, "protein_RNAP":100}
    
    return [S1,S2,S3,So1],ic
    

In [4]:
comp,ic = GeneletGate("AND1","P")
M = Mixture(name = "Switch_test", components = comp, parameter_file = "default_parameters.txt")
repr(M)
CRN = M.compile_crn()
print(CRN)


Species = AND1_INP1_OFF, rna_AND1_out_A, rna_AND1_A1, rna_AND1_I1, protein_RNAP, AND1_INP2_OFF, rna_AND1_A2, rna_AND1_I2, AND1_OUT_OFF, rna_P, rna_AND1_out_I, AND1_SOU, complex_AND1_SOU_protein_RNAP
Reactions = [
	AND1_INP1_OFF + rna_AND1_A1 --> complex_AND1_INP1_ON        massaction: k_f(AND1_INP1_OFF,rna_AND1_A1)=0.0394*AND1_INP1_OFF*rna_AND1_A1
	complex_AND1_INP1_ON + rna_AND1_I1 --> AND1_INP1_OFF + complex_AND1_INP1_AI        massaction: k_f(complex_AND1_INP1_ON,rna_AND1_I1)=0.696*complex_AND1_INP1_ON*rna_AND1_I1
	rna_AND1_A1 + rna_AND1_I1 --> complex_AND1_INP1_AI        massaction: k_f(rna_AND1_A1,rna_AND1_I1)=0.696*rna_AND1_A1*rna_AND1_I1
	complex_AND1_INP1_ON + protein_RNAP --> complex_AND1_INP1_ON + rna_AND1_out_A + protein_RNAP        massaction: k_f(complex_AND1_INP1_ON,protein_RNAP)=2.471*complex_AND1_INP1_ON*protein_RNAP
	AND1_INP1_OFF + protein_RNAP --> AND1_INP1_OFF + rna_AND1_out_A + protein_RNAP        massaction: k_f(AND1_INP1_OFF,protein_RNAP)=0.0667*AND1_INP1_OFF*pro

In [5]:
timepoints = np.linspace(0, 40, 1000)
R = CRN.simulate_with_bioscrape(timepoints, initial_condition_dict = ic)

bokeh.io.output_notebook()
p = bokeh.plotting.figure(plot_width=300, plot_height=300)
p.circle(timepoints, R["AND1_INP1_OFF"], legend_label = "OFF switch 1", color = "orange")
p.circle(timepoints, R["complex_AND1_INP1_ON"], legend_label = "ON switch 1", color = "red")
p.circle(timepoints, R["complex_AND1_INP1_AI"],legend_label = "AI complex 1", color = "green")

s = bokeh.plotting.figure(plot_width=300, plot_height=300)
s.circle(timepoints, R["AND1_INP2_OFF"], legend_label = "OFF switch 1", color = "orange")
s.circle(timepoints, R["complex_AND1_INP2_ON"], legend_label = "ON switch 2" , color = "red")
s.circle(timepoints, R["complex_AND1_INP2_AI"],legend_label = "AI complex 2", color = "green")

q = bokeh.plotting.figure(plot_width=300, plot_height=300)
q.circle(timepoints, R["AND1_OUT_OFF"], legend_label = "OFF switch 3", color = "orange")
q.circle(timepoints, R["complex_AND1_OUT_ON"], legend_label = "ON switch 3" , color = "red")

r = bokeh.plotting.figure(plot_width=300, plot_height=300)
r.circle(timepoints, R["rna_AND1_out_I"], legend_label = "Inhibitor 3", color = "blue")
r.circle(timepoints, R["complex_AND1_OUT_AI"], legend_label = "AI complex 3" , color = "green")
#p.circle(timepoints, R["complex_rna_A3_rna_I3"],legend_label = "AI complex 3", color = "green")
bokeh.io.show(row(p, s, q, r))
warnings.filterwarnings("ignore")

/home/nazmus/anaconda3/lib/python3.7/site-packages/html5lib/_trie/_base.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/home/nazmus/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/nazmus/anaconda3/lib/python3.7/site-packages/biocrnpyler-0.2.1-py3.7.egg/biocrnpyler/chemical_reaction_network.py:1089: UserWarning: The following species are uninitialized and their value has been defaulted to 0: complex_AND1_INP1_ON, complex_AND1_INP1_AI, complex_AND1_INP2_ON, complex_AND1_INP2_AI, complex_AND1_OUT_ON, complex_AND1_OUT_AI, 


Loading BokehJS ...

In [6]:
S1_off = Species("Sw1")
S2_off = Species("Sw2")
S3_off = Species("Sw3")
So1_on = Species("So1")
R = Species("I3",material_type="rna")
S1 = Switch(S1_off, transcript = "A3", activator = "A1", inhibitor = "I1" )
S2 = Switch(S2_off, transcript = "A3", activator = "A2", inhibitor = "I2" )
S3 = Switch(S3_off, transcript = "P", activator = "A3", inhibitor = "I3" )
So1 = Source(So1_on, transcript = "I3")
M = Mixture(name = "Switch_test", components = [S1,S2,S3,So1], parameter_file = "default_parameters.txt")
rxn = Reaction([R],[], k=0.9)
repr(M)
CRN = M.compile_crn()
CRN.add_reactions([rxn])
print(CRN.pretty_print())

Species (13) = {0. Sw1_OFF, 1. rna[A3], 2. rna[A1], 3. rna[I1], 4. protein[RNAP], 5. Sw2_OFF, 6. rna[A2], 7. rna[I2], 8. Sw3_OFF, 9. rna[P], 10. rna[I3], 11. So1, 12. complex[So1:protein[RNAP]]}
Reactions (18) = [
0. Sw1_OFF + rna[A1] --> complex[Sw1_OFF:rna[A1]]        
        massaction: k_f(Sw1_OFF,rna[A1])=0.0394*Sw1_OFF*rna[A1]
1. complex[Sw1_OFF:rna[A1]] + rna[I1] --> Sw1_OFF + complex[rna[I1]:rna[A1]]        
        massaction: k_f(complex[Sw1_OFF:rna[A1]],rna[I1])=0.696*complex[Sw1_OFF:rna[A1]]*rna[I1]
2. rna[A1] + rna[I1] --> complex[rna[I1]:rna[A1]]        
        massaction: k_f(rna[A1],rna[I1])=0.696*rna[A1]*rna[I1]
3. complex[Sw1_OFF:rna[A1]] + protein[RNAP] --> complex[Sw1_OFF:rna[A1]] + rna[A3] + protein[RNAP]        
        massaction: k_f(complex[Sw1_OFF:rna[A1]],protein[RNAP])=2.471*complex[Sw1_OFF:rna[A1]]*protein[RNAP]
4. Sw1_OFF + protein[RNAP] --> Sw1_OFF + rna[A3] + protein[RNAP]        
        massaction: k_f(Sw1_OFF,protein[RNAP])=0.0667*Sw1_OFF*protein[RN

In [7]:
io = {"Sw1_OFF": 500, "rna_A1": 700, "rna_I1": 200, "Sw2_OFF": 500, "rna_A2": 700, "rna_I2": 200, "Sw3_OFF": 500, "rna_A3": 0, "rna_I3": 0, "So1":170, "protein_RNAP":100}
timepoints = np.linspace(0, 40, 1000)
R = CRN.simulate_with_bioscrape(timepoints, initial_condition_dict = io)
#plt.plot(timepoints, R["rna_A3"], label = "Product: A3", color = "red")
#plt.plot(timepoints, R["dna_S1"], label = "Switch OFF", color = "green")
#plt.plot(timepoints, R["complex_A1_dna_S1"], label = "Switch ON", color = "magenta")
#plt.plot(timepoints, R["complex_A1_I1"], label = "AI complex", color = "blue")
#plt.legend()

bokeh.io.output_notebook()
p = bokeh.plotting.figure(plot_width=300, plot_height=300)
#p.circle(timepoints, R["rna_A3"], legend_label = "Activator 3")
p.circle(timepoints, R["Sw1_OFF"], legend_label = "OFF switch 1", color = "orange")
p.circle(timepoints, R["complex_Sw1_ON"], legend_label = "ON switch 1", color = "red")
#p.circle(timepoints, R["complex_Sw1_AI"],legend_label = "AI complex 1", color = "green")
#p.legend.location = "center_right"
s = bokeh.plotting.figure(plot_width=300, plot_height=300)
s.circle(timepoints, R["Sw2_OFF"], legend_label = "OFF switch 1", color = "orange")
s.circle(timepoints, R["complex_Sw2_ON"], legend_label = "ON switch 2" , color = "red")
#s.circle(timepoints, R["complex_Sw2_AI"],legend_label = "AI complex 2", color = "green")
#s.legend.location = "center_right"

q = bokeh.plotting.figure(plot_width=300, plot_height=300)
q.circle(timepoints, R["Sw3_OFF"], legend_label = "OFF switch 3", color = "orange")
q.circle(timepoints, R["complex_Sw3_ON"], legend_label = "ON switch 3" , color = "red")
#q.legend.location = "center_right"

r = bokeh.plotting.figure(plot_width=300, plot_height=300)
r.circle(timepoints, R["rna_I3"], legend_label = "Inhibitor 3", color = "blue")
r.circle(timepoints, R["complex_Sw3_AI"], legend_label = "AI complex 3" , color = "green")
#p.circle(timepoints, R["complex_rna_A3_rna_I3"],legend_label = "AI complex 3", color = "green")
bokeh.io.show(row(p, s, q, r))
warnings.filterwarnings("ignore")

/home/nazmus/anaconda3/lib/python3.7/site-packages/biocrnpyler-0.2.1-py3.7.egg/biocrnpyler/chemical_reaction_network.py:1089: UserWarning: The following species are uninitialized and their value has been defaulted to 0: complex_Sw1_ON, complex_Sw1_AI, complex_Sw2_ON, complex_Sw2_AI, complex_Sw3_ON, complex_Sw3_AI, 


Loading BokehJS ...

In [ ]:
#bokeh.io.output_notebook()
#p = bokeh.plotting.figure()
#p.circle(timepoints, R["rna_I3"], legend_label = "Inhibitor 3", color = "blue")
#p.circle(timepoints, R["complex_rna_A3_rna_I3"],legend_label = "AI complex 3", color = "green")
#bokeh.io.show(p)